In [1]:
%%bash
c++ -g -o _residual.dylib residual.cpp -fPIC -shared \
    -L$HOME/miniconda3/envs/iebeam/lib -I$HOME/miniconda3/envs/iebeam/include;
    
install_name_tool -add_rpath $HOME/miniconda3/envs/iebeam/lib _residual.dylib

In [2]:
import numpy as np
import scipy


import cffi
from scipy.special import legendre
from scipy.optimize import newton_krylov, fsolve, anderson


ffi = cffi.FFI()


ffi.cdef("""
            void computeResidual(const double youngs_mod, const double area, const double moment_of_inertia, 
                       const int int_rule, const double* int_points, const double* int_wts, 
                       const int num_elements, const double* nodes, const double* uknowns, 
                       double* residual);
            """)

res = ffi.dlopen("_residual.dylib")

In [7]:
class InextensibleBeamFEA(object):
    
    def __init__(self, number_of_elements=1, length=1, E=1, I=1, A=1, P2=1, int_rule=3):
    
        self.number_of_elements = number_of_elements
        self.length = length
        
        self.E = E
        self.I = I
        self.A = A
        
        self.P2 = P2
        
        self.int_rule = int_rule
        self.xi, self.int_wts = np.polynomial.legendre.leggauss(int_rule)
        
        self.nodes = np.linspace(0, length, num=(number_of_elements * 2 + 1))
        
        number_of_midpoint_nodes = (self.nodes.shape[0]) // 2
        self.unknowns = np.zeros((number_of_midpoint_nodes + 1) * 4 + number_of_midpoint_nodes)
        self.residual = np.zeros_like(self.unknowns)
        
    def compute_residual(self, X):
        
        #Compute the residual from C library
        res.computeResidual(self.E , self.A, self.I, self.int_rule, ffi.from_buffer(self.xi), 
                            ffi.from_buffer(self.int_wts), self.number_of_elements, ffi.from_buffer(self.nodes), 
                            ffi.from_buffer(X), ffi.from_buffer(self.residual))
        
        #Zero out boundary conditions in residual (cantileaver beam)
        #self.residual[0] = X - 0.1
        
        #Apply load to end of beam
        self.residual[-3] -= self.P2
        
        return self.residual
    
    
    def solve(self):
        
        guess = np.zeros_like(self.unknowns)
        
        return newton_krylov(self.compute_residual, guess)

In [8]:
problem = InextensibleBeamFEA(number_of_elements=1, E=200e9, I=0.2 ** 4. / 12., A=0.2*0.2, length=10, P2 = 1e6)

problem.solve()

NoConvergence: [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.49011612e-11   0.00000000e+00
   0.00000000e+00]